In [3]:
from datasets import load_metric
import pickle5 as pickle
import os
import re
import numpy as np

def compute_measure_no_impossible(category, indices):
    sub_df = s.loc[indices, :]
    pred = []
    true = []
    for row in sub_df.itertuples():
        try:
            pred.append({"id": str(row.Index), "prediction_text": row.predictions})
        except:
            pred.append("")
   
        true.append({"id": str(row.Index), "answers": {'answer_start': [1 for i in range(len(row.gold_answers))], 'text': row.gold_answers}})  
    
    #return len(sub_df), metric.compute(predictions=pred, references=true)
    
    print(f'Number of questions in \"{category}\" category: {len(sub_df)}')
    print(metric.compute(predictions=pred, references=true))

def compute_measure_with_impossible(category, indices):
    sub_df = s.loc[indices, :]
    pred = []
    true = []
    for row in sub_df.itertuples():
        if len(str(row.predictions)) == 0:
            pred.append({"id": str(row.Index), "prediction_text": "", 'no_answer_probability': 1.})
        else:
            pred.append({"id": str(row.Index), "prediction_text": row.predictions, 'no_answer_probability': 0.})
      
        if len(row.gold_answers) == 0:
            true.append({"id": str(row.Index), "answers": {'answer_start': [1], 'text': []}})
        else:
            true.append({"id": str(row.Index), "answers": {'answer_start': [1 for i in range(len(row.gold_answers))], 'text': row.gold_answers}})  

    print(f'Number of questions in \"{category}\" category: {len(sub_df)}')
    results = metric.compute(predictions=pred, references=true)
    print(f"EM: {np.round(results['exact'], 2)} F1: {np.round(results['f1'], 2)}")  
    
metric = load_metric('squad_v2')

In [6]:
pred_file_path = os.path.abspath('../../zero_shot_predictions/techqa/navteca_roberta-base-squad2_Saptarshi7_techqa-squad-style_predictions.pkl')
s = pickle.load(open(pred_file_path, 'rb'))

what_indices = []
why_indices = []
when_indices = []
who_indices = []
which_indices = []
where_indices = []
how_indices = []
is_indices = []
in_indices = []
other_indices = []

for row in s.itertuples():
    if re.search(r'\bwhat\b', row.question, re.IGNORECASE):
        what_indices.append(row.Index)
    elif re.search(r'\bwhy\b', row.question, re.IGNORECASE):
        why_indices.append(row.Index)
    elif re.search(r'\bwhen\b', row.question, re.IGNORECASE):
        when_indices.append(row.Index)
    elif re.search(r'\bwho\b', row.question, re.IGNORECASE):
        who_indices.append(row.Index)
    elif re.search(r'\bwhich\b', row.question, re.IGNORECASE):
        which_indices.append(row.Index)
    elif re.search(r'\bwhere\b', row.question, re.IGNORECASE):
        where_indices.append(row.Index)
    elif re.search(r'\bhow\b', row.question, re.IGNORECASE):
        how_indices.append(row.Index)
    elif re.search(r'\bis\b', row.question, re.IGNORECASE):
        is_indices.append(row.Index)
    elif re.search(r'\bin\b', row.question, re.IGNORECASE):
        in_indices.append(row.Index)
    else:
        other_indices.append(row.Index)
    
if len(what_indices) != 0:
    compute_measure_with_impossible('what', what_indices)
if len(why_indices) != 0:
    compute_measure_with_impossible('why', why_indices)
if len(when_indices) != 0:
    compute_measure_with_impossible('when', when_indices)
if len(who_indices) != 0:
    compute_measure_with_impossible('who', who_indices)
if len(which_indices) != 0:
    compute_measure_with_impossible('which', which_indices)
if len(where_indices) != 0:
    compute_measure_with_impossible('where', where_indices)
if len(how_indices) != 0:
    compute_measure_with_impossible('how', how_indices)
if len(is_indices) != 0:
    compute_measure_with_impossible('is', is_indices)
if len(in_indices) != 0:
    compute_measure_with_impossible('in', in_indices)
if len(other_indices) != 0:
    compute_measure_with_impossible('other', other_indices)

Number of questions in "what" category: 47
EM: 0.0 F1: 4.08
Number of questions in "why" category: 32
EM: 9.38 F1: 12.61
Number of questions in "when" category: 54
EM: 0.0 F1: 0.84
Number of questions in "which" category: 9
EM: 0.0 F1: 6.24
Number of questions in "where" category: 45
EM: 4.44 F1: 6.1
Number of questions in "how" category: 36
EM: 0.0 F1: 2.45
Number of questions in "is" category: 42
EM: 2.38 F1: 7.21
Number of questions in "in" category: 12
EM: 0.0 F1: 0.0
Number of questions in "other" category: 33
EM: 0.0 F1: 2.65


In [ ]:
metric

In [ ]:
x = []
for row in s.loc[other_indices, :].itertuples():
    if re.search(r'\bin\b', row.question, re.IGNORECASE):
        x.append(row.Index)
compute_measure('in', x)

In [ ]:
s.loc[other_indices, :].question.to_list()

In [ ]:
s.loc[in_indices, :].question.to_list()